# To do functions:
* Check margin limit breach: margin_breached() -> (breached, breached_value) 
* Make underlying contracts: make_und_contracts(chains_nse.pkl)
* Get the latest prices: get_prices(contracts) -> {conId: price}
* Get uncovered calls: cancel_nakeds()
* Get current positions (df_position) {get_positions}
* Filter functions {maskFR, mask52, maskPutsOnly, masksd_LoHi}
* Check uncovered shorts and longs (df_uncovered) {shorts_longs}

In [ ]:
# imports
from z_helper import *
util.startLoop()

ib = get_connected('nse', 'live')

# from json
a = assign_var('nse') + assign_var('common')
for v in a:
    exec(v)
    
jup_disp_adjust() # adjust jupyter's display

with open(logpath+'test.log', 'w'):
    pass # clear the run log

util.logToFile(logpath+'test.log')

In [ ]:
# input files for the functions
df_chains = pd.read_pickle(fspath+'chains.pkl')
df_sized = pd.read_pickle(fspath+'sized_nse.pkl')
assignment_limit = nse_assignment_limit

In [ ]:
# Preparatory inputs for df_sized
df_chains = pd.read_pickle(fspath+'chains.pkl')
df_ohlcs = pd.read_pickle(fspath+'ohlcs.pkl')

filt1 = "sdmult" # fr=fallrise*, sdmult=standard deviation, , hilo52 = hilo52
filt2 = "all" # all=all options*, putsonly = puts only

In [ ]:
%%time
#... Remove chains not meeting put and call std filter

# get dte and remove those greater than maxdte
df_chains = df_chains.assign(dte=df_chains.expiry.apply(get_dte))                    
df_chains = df_chains[df_chains.dte <= maxdte]

# replace dte with 1 for dte <= 0
df_chains.loc[df_chains.dte <=0,  'dte'] = 1

# generate std dataframe
df = df_ohlcs[['symbol', 'stDev']]  # lookup dataframe
df = df.assign(dte=df.groupby('symbol').cumcount()) # get the cumulative count for location as dte
df.set_index(['symbol', 'dte'])

df1 = df_chains[['symbol', 'dte']]  # data to be looked at
df2 = df1.drop_duplicates()  # remove duplicates

df_std = df2.set_index(['symbol', 'dte']).join(df.set_index(['symbol', 'dte']))

# join to get std in chains
df_chainstd = df_chains.set_index(['symbol', 'dte']).join(df_std).reset_index()

# first stocks
und_contracts = [Stock(symbol, exchange=exchange) for symbol in df_chainstd.symbol.unique()]
und_quals = ib.qualifyContracts(*und_contracts)

# then indexes
idxs = set(df_chainstd.symbol.unique())-{u.symbol for u in und_quals}
und_idxs = [Index(symbol, exchange=exchange) for symbol in idxs]
und_idx_quals = ib.qualifyContracts(*und_idxs)

und_quals = und_idx_quals+und_quals

tickers = ib.reqTickers(*und_quals)

uprice_dict = {u.contract.conId: u.marketPrice() for u in tickers}

df_chainstd = df_chainstd.assign(undPrice=df_chainstd.undId.astype('int').map(uprice_dict))

# make puts and calls dataframe with std filter
df_puts = df_chainstd[df_chainstd.strike < (df_chainstd.undPrice-(df_chainstd.stDev*putstdmult))]
df_puts = df_puts.assign(right = 'P')

df_calls = df_chainstd[df_chainstd.strike > (df_chainstd.undPrice+(df_chainstd.stDev*callstdmult))]
df_calls = df_calls.assign(right = 'C')

df_opt = pd.concat([df_puts, df_calls], sort=False).reset_index(drop=True)

# get lo52 and hi52
df_opt = df_opt.set_index('symbol').join(df_ohlcs.groupby('symbol')
                                         .close.agg(['min', 'max'])
                                         .rename(columns={'min': 'lo52', 'max': 'hi52'})).reset_index()

# make (df and dte) tuple for fallrise
tup4fr = [(df_ohlcs[df_ohlcs.symbol == s.symbol], s.dte) 
          for s in df_opt[['symbol', 'dte']].drop_duplicates().itertuples()]

# get the fallrise and put it into a dataframe
fr = [fallrise(*t) for t in tup4fr]
df_fr = pd.DataFrame(fr, columns=['symbol', 'dte', 'fall', 'rise' ])

# merge with df_opt
df_opt = pd.merge(df_opt, df_fr, on=['symbol', 'dte'])

# make reference strikes from filter
if filt1 == 'sdmult':
    df_opt = df_opt.assign(strikeRef = np.where(df_opt.right == 'P', 
                                            df_opt.undPrice-putstdmult*df_opt.stDev, 
                                            df_opt.undPrice+callstdmult*df_opt.stDev))
elif filt1 == 'hilo52':
    df_opt = df_opt.assign(strikeRef = np.where(df_opt.right == 'P', 
                                            df_opt.lo52 - df_opt.stDev, 
                                            df_opt.hi52 + df_opt.stDev))    
else: # default (fallrise)
    df_opt = df_opt.assign(strikeRef = np.where(df_opt.right == 'P', 
                                                df_opt.undPrice-df_opt.fall, 
                                                df_opt.undPrice+df_opt.rise))    

In [ ]:
df_opt[(df_opt.symbol == 'APOLLOTYR') & (df_opt.dte == 20) & (df_opt.right == 'C')].head()

In [ ]:
df_ohlcs[df_ohlcs.symbol=='NIFTY50'].close.rolling(20).mean().iloc[20]

In [ ]:
df_acc=df_ohlcs[df_ohlcs.symbol=='ACC'].loc[:252/2]

In [ ]:
df_acc.close.rolling(20).mean()

In [ ]:
66.4+27.10

In [ ]:
df_chains = pd.read_pickle(fspath+'chains_nse.pkl')

#...undprice refresh
syms = list(df_chains.symbol.unique())

# get the contracts and undprice
qual_stks = ib.qualifyContracts(*[Stock(s, exchange=exchange) for s in syms])
idxs = {s for s in syms if s not in {q.symbol for q in qual_stks}}
qual_idxs = ib.qualifyContracts(*[Index(s, exchange=exchange) for s in idxs])
qual_unds = qual_stks+qual_idxs
undtickers = ib.reqTickers(*qual_unds)
uprice_dict = {u.contract.conId: u.marketPrice() for u in undtickers}